In [1]:
SpecialToken = r"""#(*+/\:;<=[^_₩{|~‘“""" # 띄우기
NotSpecialToken = r""",'"]}>)”’"""   # 붙이기
                            # 여기에 없으면 유지
exclude = set(SpecialToken)
unexclude = set(NotSpecialToken)

In [2]:
# 특수문자 제거 함수
def str2token(TokenList):
    NewTokenList = []

    for i in TokenList:
        if i in exclude:
            TokenList = TokenList.replace(i, ' ')

        elif i in unexclude:
            TokenList = TokenList.replace(i, '')

    # NewTokenList = TokenList.split()
    return TokenList


In [144]:
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
import os
import numpy as np

import json
import random
from collections import Counter

from konlpy.tag import Komoran
komoran = Komoran()

os.chdir('../')
import yake
from yake.korea_token import edit_josa, edit_sentences
os.chdir('test_dacon/')

from sklearn.feature_extraction.text import TfidfVectorizer 
from collections import defaultdict

from krwordrank.word import KRWordRank
import math

from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pandas import DataFrame
from test_testmain import RougeScorer

import warnings
warnings.filterwarnings(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# stopword.txt 읽어오기 위한 path 및 파일이름
language = 'ko'
path = '../yake/StopwordsList/'
txt = 'stopwords_{}.txt'.format(language)

try:
    f = open(path + txt, encoding='UTF-8')
    ExistStopwords = set(f.read().split())
    f.close()
except FileNotFoundError:
    ExistStopwords = [None]

ExistStopwords = list(set(ExistStopwords))

In [5]:
with open("valid_original.json", "r", encoding="utf8") as f:
    contents = f.read() # string 타입
    json_data = json.loads(contents)
    f.close()

In [31]:
def kyake(txt:list):
    # print("txt: ", txt)
    kw_extractor = yake.KeywordExtractor(n=[1,2,3],top=50, stoplen=2, windowsSize=1, WG=False, ReDup=False)
    return_tuple = kw_extractor.extract_keywords(''.join(txt))
    # print("return_tuple: ", return_tuple)
    return_list = [i[0] for i in return_tuple]
    return_list = return_list[:10]
    # print("return_list: ", return_list)

    return return_list

In [17]:
def tfidf_parser(txt:list):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(txt)
    matrix = vectorizer.transform(txt)

    # vocabulary_word_id = defaultdict(int)
    vocabulary_word_id = {token:idx for idx,token in enumerate(vectorizer.get_feature_names())}
        
    # 특징 추출 결과: {"token": value}
    result = {token:matrix[0, vocabulary_word_id[token]] for token in vectorizer.get_feature_names()}
        
    # 내림차순 (중요도 high) 기준 정렬
    result = sorted(result.items(), key = lambda item: item[1], reverse = True)

    return_list = [i[0] for i in result if i[1] != 0 and i[0] not in ExistStopwords]
    return_list = return_list[:10]

    return return_list

In [7]:
news_list = np.random.choice(range(0,30120),1000,replace=False)  # max: 30122 || 30122개의 뉴스 데이터 # random.randint(0, 30000)

# * 키워드가 1-gram일 때 합성어, 고유명사만 출력 및 조사 떼는 실험

In [ ]:
def deljosa(word:str):
        '''
        한국어 조사와 어말어미의 출현빈도 조사
        상위 70개 통합 조사의 상대적 출현빈도
        KT Data Set 99.8% 커버 조사
        '''
        if word[-4:] in ['으로부터',"으로서의"]:
            return word[:-4]
        elif word[-3:] in ['에서는','으로써','에서의','로부터','으로는',
                        "에서도","까지의","이라는","으로의","이라고"
                        ,"보다는","로서의","만으로"]:
            return word[:-3]
        elif word[-2:] in ['으로','에서','하고','이다','과를','보다','에는'
                        ,'이나','로서','에게','까지','과는','만을',"대로"
                        ,"이고","에도","과의","로의","이며","로써","로는"
                        ,"만이","와의","마다","와는","이라","에만","라고"
                        ,"처럼","부터","로도",'토록']:
            return word[:-2]
        elif word[-1:] in ['을',"의","를","에","은","이","는","과","로","가"
                        ,"와","고","여","도","다","든","라","나","만","며"
                        ,"께","요"]:
            return word[:-1]
        #print( word[:-3])
        return word
def strainer(txt):
    return_number = []
    return_value = []

    for i in range(len(txt)):
        if ' ' not in txt[i]:
            if komoran.pos(txt[i])[0][1] in ['NNG','NR','NP']:
                if len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['NNG','NNP','NNB','NR','NP']:
                    # return_value.append(komoran.pos(txt[i])[0][0] + komoran.pos(txt[i])[1][0])
                    return_number.append(i)
            
            elif komoran.pos(txt[i])[0][1] in ['NNP']:
                return_number.append(i)

            elif komoran.pos(txt[i])[0][1] in ['MM']:
                if len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['NNG','NNP','NNB','NR','NP']:
                    return_number.append(i)

            elif komoran.pos(txt[i])[0][1] == 'XPN':
                if len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['NNG','NNP','NR','NP']:
                    return_number.append(i)
                elif len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['VV', 'VA', 'VX', 'VC', 'XR']:
                    if len(komoran.pos(txt[i]))>2 and komoran.pos(txt[i])[2][1] in ['ETN']:
                        if len(komoran.pos(txt[i]))>3 and komoran.pos(txt[i])[3][1] in ['NNG']:
                            return_number.append(i)

            elif komoran.pos(txt[i])[0][1] == 'XR':
                if len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['XSN', 'NNG','NR','NP']:
                    return_number.append(i)

            elif komoran.pos(txt[i])[0][1] == 'NA':
                return_number.append(i)

            elif komoran.pos(txt[i])[0][1] in ['SL', 'SH']:
                if len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['NNG','NNP','NNB','NR','NP']:
                    return_number.append(i)      
                elif len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['VV', 'VA', 'VX', 'VC', 'XR']:
                    if len(komoran.pos(txt[i]))>2 and komoran.pos(txt[i])[2][1] in ['ETN']:
                        if len(komoran.pos(txt[i]))>3 and komoran.pos(txt[i])[3][1] in ['NNG']:
                            return_number.append(i)

            elif komoran.pos(txt[i])[0][1] in ['SN']:
                if len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['NNG','NNP','NNB','NR','NP']:
                    return_number.append(i)
                elif len(komoran.pos(txt[i]))>1 and komoran.pos(txt[i])[1][1] in ['SW']:
                    return_number.append(i)
                elif len(komoran.pos(txt[i]))>2 and komoran.pos(txt[i])[1][1] in ['SF']:
                    if len(komoran.pos(txt[i]))>3 and komoran.pos(txt[i])[2][1] in ['SN']:
                        if len(komoran.pos(txt[i]))>4 and komoran.pos(txt[i])[3][1] in ['SW']:
                            return_number.append(i)
                        elif len(komoran.pos(txt[i]))>4 and komoran.pos(txt[i])[3][1] in ['NNG','NNP','NNB','NR','NP']:
                            return_number.append(i)
        else:
            return_number.append(i)

    # print('return_number: ', return_number)
    for i in return_number:
        return_value.append(txt[i])
    # print("return_value: ", return_value)

    return return_value

In [145]:
def kyake(txt:list):
    # print("txt: ", txt)
    kw_extractor = yake.KeywordExtractor(n=[1,2,3],top=50, stoplen=2, windowsSize=1, WG=False, ReDup=False)
    return_tuple = kw_extractor.extract_keywords(''.join(txt))
    # print("return_tuple: ", return_tuple)

    return_list = [i[0] for i in return_tuple]
    return_list = [deljosa(i) for i in strainer(return_list)]
    return_list = [i for i in return_list if len(i) > 1]
    
    return_list = return_list[:10]
    # print("return_list: ", return_list)

    return return_list

In [150]:
NEWS = np.random.choice(range(0,30120),30000,replace=False)  # max: 30122 || 30122개의 뉴스 데이터 # random.randint(0, 30000)

In [151]:
df_trainset = DataFrame(columns=["id", "summary"])
df_testset = DataFrame(columns=["id", "summary"])

scorer = RougeScorer()
_dataframe = DataFrame()

R1 = []
R2 = []
Rl = []
Rw = []

for i in tqdm(NEWS):
    
    id = None
    topic_number = None
    
    text_list = [
        str2token(json_data["documents"][i]["text"][j][k]["sentence"] + '.') 
        for j in range(len(json_data["documents"][i]["text"])) 
        for k in range(len(json_data["documents"][i]["text"][j]))
    ]

    topic_number = json_data["documents"][i]["extractive"]
    id = [int(json_data["documents"][0]['id'])]

    text = [' '.join(text_list)]
    STRTexT = ' '.join(text_list)
    LISTText = text_list

    new_text = edit_sentences(STRTexT)
    total_value = [' '.join(edit_josa(new_text[x])) for x in range(len(new_text))]
    TUNNEDText = total_value

    ### Kyake 키워드 추출: kyake(text)                     || txt:list
    keywords = kyake(text)

    score = 0
    for l in range(len(text_list)):         # text_list[l] : text_list의 l번째 문장
        for m in range(len(keywords)):      # keywords : text파일의 문서를 각 함수들을 돌린 후 도출되는 키워드들.
            if keywords[m] in text_list[l]:
                score = score + 1

        score_data = dict(zip(["id", "index", "score"], [i, l, score]))
        score = 0
        _dataframe = _dataframe.append(score_data, ignore_index=True)

    multi_index = _dataframe.sort_values(by=['score', 'index'], ascending=[False, True]).groupby("id").head(3)
    multi_index = multi_index.sort_values(by=['id','score'], ascending=[True, False])

    summary_number = [int (i) for i in list(multi_index[multi_index['id']==i]['index'])]

    try:    
        df_testset['id'] = id
        df_testset['summary'] = [[text_list[int(summary_number[0])],text_list[int(summary_number[1])], text_list[int(summary_number[2])]]]
        df_trainset['id'] = id
        df_trainset['summary'] = [[text_list[int(topic_number[0])],text_list[int(topic_number[1])], text_list[int(topic_number[2])]]]
    except TypeError:
        print("TypeError 발생: ", TypeError)
    
    score_set = scorer.compute_rouge(df_trainset, df_testset)
    rouge_1 = score_set[0]
    rouge_2 = score_set[1]
    rouge_l = score_set[2]
    rouge_w = score_set[3]

    R1.append(rouge_1)
    R2.append(rouge_2)
    Rl.append(rouge_l)
    Rw.append(rouge_w)

    # print("키워드들: ", keywords)
    # print(i,"번째 문단..")
    # print("선택 문장 : ", (summary_number))
    # print([text_list[int(summary_number[0])],text_list[int(summary_number[1])], text_list[int(summary_number[2])]])
    # print("정답지 : ", topic_number)
    # print([text_list[int(topic_number[0])],text_list[int(topic_number[1])], text_list[int(topic_number[2])]])
    # print()

print("rouge_1의 평균점수: ", sum(R1) / len(R1))
print("rouge_2의 평균점수: ", sum(R2) / len(R2))
print("rouge_l의 평균점수: ", sum(Rl) / len(Rl))
print("rouge_w의 평균점수: ", sum(Rw) / len(Rw))

100%|██████████| 30000/30000 [18:22:22<00:00,  2.20s/it]       

rouge_1의 평균점수:  0.474906216848075
rouge_2의 평균점수:  0.351124266929438
rouge_l의 평균점수:  0.43411827951858845
rouge_w의 평균점수:  0.2715256369642409
